In [1]:
from collections import defaultdict, Counter
import json
from itertools import product

In [2]:
journey_rawdata = open("data/journey.csv").readlines()
print(journey_rawdata[:5])

['1624,9\n', '1624,5\n', '1624,9\n', '1624,40\n', '1624,40\n']


In [15]:
groups = json.load(open("data/event_groupings.json"))

In [4]:
journeys = defaultdict(list)
for line in journey_rawdata:
    customer, event = line.split(',')
    event = groups["Type_to_group"][event[:-1]]
    customer = int(customer)
    journeys[customer].append(event)

In [5]:
journey_keys = iter(journeys.keys())
journeys[next(journey_keys)]

[0, 4]

In [6]:
journeys[next(journey_keys)]

[0, 4, 2, 4, 2]

In [7]:
def compress(events):
    compressed = []
    for i, event in enumerate(events):
        if event == 7 and events[i-1] == 7:
            continue
        if i > 0 and event == 0 and events[i-1] == 0:
            continue
        else:
            compressed.append(event)
    return compressed

In [8]:
def detect_proxy_cx(events):
    # via CC
    if 3 in events and 0 not in events[events.index(3):] and 1 not in events[events.index(3):]:
        events.append(-1)
        events[events.index(3)+1] = 15
    # via delay        
    if 5 in events and 0 not in events[events.index(5):] and 1 not in events[events.index(5):]:
        events.append(-1)
        events[events.index(5)+1] = 15
    return events

In [9]:
def pairs(events, length_cap):
    events = compress(events)
    events = detect_proxy_cx(events)
    events = [(0, x[1]) if x[1] in [2, 15] else x for x in enumerate(events[:length_cap])]  # only 1 cancellation event
    if (0, 2) in events:
        events = events[:events.index((0,2)) + 1]
    if (0, 15) in events:
        events = events[:events.index((0,15)) + 1]
    return [(events[i], events[i+1]) for i in range(len(events) - 1)]

# >> pairs([1,5,5,27,3],2)
# [((0, 1), (1, 5))]

1. get set of pairs that actually occur - looks like all or close. XX
1. output to a file and record indices, ie create dict{pair: index}
1. create counter of union of journeys: dict(pair of pairs : count) XX
1. use lookup dict to map keys to dict(pair of indices : count)
1. output

What about subscription status? What can we glean from this file?

Definitely need to create groupings of event_types.

```
enumerate the possible nodes (0-20, 0-13), 260 of them. Gives:
(index, (event_number, event_group))
create func to create json for (index, (event_clock, event_group)) -> [{name: groups["Groups"][event_group]}]
create dict:
{(event_number, event_group): index}
use dict to map edges:
(((num, grp), (num, grp)), count) -> {source: index, target: index, value: count}
```

In [10]:
def endpoints(segments):
    endpoints = set()
    for seg in segments:
        endpoints.add(seg[0])
        endpoints.add(seg[1])
    return endpoints

In [21]:
num_events = 10

journey_segments = []  # [((a, b), (c, d))]
nodeset = set()
for j in journeys.values():
    if j[0] == 0:
        segments = pairs(j, num_events)
        nodeset = nodeset.union(endpoints(segments))
        journey_segments.extend(segments)

In [22]:
c = Counter(journey_segments)

In [23]:
possible_node_ids = list(enumerate([x for x in product(range(num_events), range(16)) if x in nodeset]))
nodes = [{"name": groups["Groups"][node[1][1]]} for node in possible_node_ids] #  + "-" + str(node[1][0])
node_lookup = {node[1]:node[0] for node in possible_node_ids}

In [24]:
edges = [{"source": node_lookup[k[0]], 
          "target": node_lookup[k[1]], "value": v} for k,v in c.items()]

In [25]:
graph = {"nodes": nodes, "links": edges}
json.dump(graph, open("data/graph.json", "w"), indent=2)

In [26]:
cp data/graph.json visapp/graph.json